# Part 3: Building regression models

In this section, we will use our data to build a regression model to predict the compressive strength of concrete based on the mixture.

We will be using the Scikit-Learn library, the de-facto python library for building machine learning models. It has a clean, consistent API for machine learning algorithms, and various capabilities for data preparation. More advanced users may build machine learning 'pipelines' which pass data through various transformation steps to process it finally making predictions with a model.

Building machine learning algorithms effectively can require a lot of experience; as well as model performance there are many considerations that need to be taken into account, such as how your model will be used once it is built.

If you are interested in building experience in this area, there are a number of sources of data sets, such as UC Irvine ML repository, various high-quality online courses (Coursera, EdX, Udemy etc), and some good blogs.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import Image
from IPython.core.display import HTML 

from funcs.training_fun import train_test

%matplotlib inline

We are going to start by using a model called a random forest.

This model builds a number of decision trees, where each split aims to separate the data into two groups which minimise the sum of squares error:

$$ SSE = \sum_{i \in S_1} (y_i - \bar{y}_1)^2 + \sum_{i \in S_2} (y_i - \bar{y}_2)^2 $$

where $\bar{y}_i$ is the average on the left and the right of the split, and $y_i$ are the observed values of the outcome. 

This is done by searching over every value of every feature in the group to decide which split gives the greatest reduction in error. The fitting is performed recursively until a stopping criteria is reached.

Once the trees are grown, predictions can be made. The new observation instance is scored on every tree in the forest- values are fed through the tree, and every node serves as an if/else statement to decide if the observation should go left or right. When it hits a terminal node (a 'leaf'), the average value of the training data in that node is taken as the tree's prediction. This is then averaged over the results of all the trees in the forest to produce a final prediction.

The trees are decorrelated from one another by training them on randomly sampled (bootstrapped) subsets of the training data, and at each split the set of candidate training features are also randomly selected (m are randomly chosen from n features).

This model is very powerful, and can produce very high quality predictions.

In [ ]:
Image(filename= "img/rf.jpg", width=900, height=1000)

## Preparing our data with a train/test split

One major danger when building models is to overfit to the training data. This means you have trained your model so specifically to the training data, that it will be describing the noise in the training data as well as any general behaviour. A model which is overfit will perform very well on the training data, but will generalise poorly.

To remedy this, we will split our data into an 75:25 split. We will build our model using 75% of the data, and evaluate it on the hold-out 25% of the data. This will give us an indication of the model's true performance.

In [ ]:
# let's read in the data we prepared
concrete = pd.read_csv("processed/concrete_processed.csv")

### Now we have a problem...

We want to model mixture proportions, and we currently have the ingredients by weight. We also have age in days, which is not a mixture ingredient.

Let's normalise our data, so we have ingredients by proportion instead. Pandas comes to the rescue!

In [ ]:
mixture_ingredients = ["Cement", "BlastFurnaceSlag", "FlyAsh",
                       "Water", "Superplasticizer", "CoarseAggregate",
                       "FineAggregate"]


# we divide every ingredient weight by the sum of each mixture
normalised_ingredients = (
    concrete
        .loc[:, mixture_ingredients]
        .div(concrete.sum(axis=1), axis = 0)
)


# join normalised mixture proportion to age and compressive strength
concrete_normalised = pd.merge(
    concrete.loc[:, ["Age", "CompressiveStrength"]],
    normalised_ingredients,
    left_index = True,
    right_index = True
)

In [ ]:
# you will not get the same results if you change the seed here.
# for reproducibility and consistency, you should always seed any function
# that uses a random process
train, test = train_test_split(concrete_normalised, train_size = 0.75, random_state = 3456)

print("train size: {} observations \ntest size: {} observations".format(train.shape[0], test.shape[0]))

## Model fitting

The Scikit-Learn library has a clean, consistent API- check out the docs [here](scikit-learn.org)

It contains an implementation for a random forest regression model, which we can easily fit  using the default parameters.

Most machine learning models have hyperparameters, that is, free parameters than need tuning to find the best possible model. We can start by fitting the default: a forest of 10 trees, which some may consider to be more of a shrubbery than a forest.

In [ ]:
#only set random state, to ensure reproducible results
rf_model = RandomForestRegressor(random_state = 1234)
# whar are the default options?
rf_model

In [ ]:
# lets fit our data to the default model
feature_names = concrete.columns.values.tolist()
outcome_name = "CompressiveStrength"
feature_names.remove(outcome_name)

rf_model.fit(train.loc[:, feature_names], train.loc[:, outcome_name])

### How well do we perform on our training data?

Now our model is fit to the trainig data, we can investigate its performance by plotting the predictions of compressive strength vs the actual values. We can also investigate the distribution of the errors.

Also, we can calculate the root mean square error, which will give us the average absolute error of our predictions in MPa.

In [ ]:
predictions_train = rf_model.predict(train.loc[:, feature_names])
error_train = predictions_train - train.loc[:, outcome_name].values

f, axarr = plt.subplots(1, 2, figsize = (12, 6), dpi=80)

for i in range(2):
    axarr[i].spines['right'].set_visible(False)
    axarr[i].spines['top'].set_visible(False)

    
axarr[0].scatter(predictions_train, train.loc[:, outcome_name].values, alpha = 0.6)
axarr[0].set_xlabel('prediction train')
axarr[0].set_ylabel('outcome train')
axarr[0].set_xlim(0, 90)
axarr[0].set_ylim(0, 90)

axarr[1].scatter(train.loc[:, outcome_name].values, error_train, alpha = 0.6)
axarr[1].set_xlabel('compressive strength')
axarr[1].set_ylabel('prediction train - outcome train')

f.subplots_adjust(wspace = 0.3);


In [ ]:
# what is the rmse on the training set?
print("training set rmse: {}".format(mean_squared_error( train.loc[:, outcome_name].values, predictions_train)** 0.5))

It's important to mention we see an effect called heterosketadicity here- our model's performance quality is not consistent for higher compressive strengths. This isnt suprising, as we have fewer observations for compressive strength > 60, however it is not ideal as we want to build the strongest mixture. This is something we would come back to try and remedy if we were performing a serious study. 

For instance, we could redefine the cost function to more harshly penalise errors when the true value of the mixture is > 60.

### Things are too good to be true...

If we reported these results, we are making the most basic mistake! We have likely overfit to our training data to a degree, and we should evaluate the model on our independent test data set to get an idea of how well the model generalises.

In [ ]:
predictions_test = rf_model.predict(test.loc[:, feature_names])
error_test = predictions_test - test.loc[:, outcome_name].values

f, axarr = plt.subplots(1, 2, figsize = (12, 6), dpi=80)

for i in range(2):
    axarr[i].spines['right'].set_visible(False)
    axarr[i].spines['top'].set_visible(False)

    
axarr[0].scatter(predictions_test, test.loc[:, outcome_name].values, alpha = 0.6)
axarr[0].set_xlabel('prediction test')
axarr[0].set_ylabel('outcome test')
axarr[0].set_xlim(0, 90)
axarr[0].set_ylim(0, 90)

axarr[1].scatter(test.loc[:, outcome_name].values, error_test, alpha = 0.6)
axarr[1].set_xlabel('observation number')
axarr[1].set_ylabel('prediction test - outcome test')

f.subplots_adjust(wspace = 0.3);


In [ ]:
print("test set rmse: {}".format(mean_squared_error( test.loc[:, outcome_name].values, predictions_test)** 0.5))

The test set root mean square error is higher- essentially this means on average, our predictions are $\pm$ 5MPa.
Seeing a higher RMSE for the test set isn't unexpected, as the predictions on the training data have overfit to describe the training data too specifically, and will never perform as well on unseen test data.

## Exercise: hyperparameter tuning

There is never any guarentee that the default hyperparameters are the best choice for the model. Your task is to tune the number of trees, depth of trees, and number of features to randomly select in the model.

I have provided a function which will allow you to test altering the hyperparameters, showing the resulting train and test error.

Your aim is to minimise the RMSE on the test data by experimenting values for n_trees, n_features and max_depth.

In [ ]:
# investigate how you can alter n_trees, n_featues and max_depth to beat this default model
# note that there are are only 8 features so n_features cannot exceed this
train_test(train.loc[:, feature_names], train.loc[:, outcome_name], # do not touch
           test.loc[:, feature_names], test.loc[:, outcome_name], # do not touch
           n_trees = 10, 
           n_features = 4,
           max_depth = 6)


## Cross validation

Now, this is a naive way to tune the model hyperparameters. Cross validation is often used for this purpose. There are several variations, but k-fold cross validation is most common, often used in conjunction with a grid search.

The idea is to define a number of combinations of the values of the hyperparameters, which is the grid of values we want to search over. For each combinations of hyperparameter, we split the data into k folds, where we train on $k-1$ and test on the left out fold. We then iterate through all the folds, building and evaluating the model on each combination of the training data. 

This allows us to select the 'best' model on training data, and then we can apply it to the test data set for a final validation. The model would then be ready to apply to any new data samples.

For time considerations we will not investigate now, but if anyone is interested, do investigate the GridSearchCV class in the scikit-learn library.

## As well as the RF model, we will also train a GBR model

Think of this as a complex 'black box' model. It can produce very high quality predictions when tuned properly. We dont have time to discuss now, but if you are interested to understand how boosting works, the scikit-learn documentation is a good place to start. 'AdaBoost' is probably the most intuitive boosting scheme. We focussed on the random forest model previously as it is easier to understand intuitively.

The GBR model also makes high quality predictions when tuned well, and can also be much much faster to make predictions once trained than a random forest. This is important for the mixture optimisation we will perform in the next part of this tutorial. 

In [ ]:
###################################################
###################################################
# DO NOT RUN ANY OF THE BELOW: THIS IS PURELY FOR REFERENCE
# IF YOU ARE INTERESTED TO FOLLOW UP AFTER THIS TUTORIAL
###################################################
###################################################

# import pickle

# rf_model_final = (
#     RandomForestRegressor(random_state = 1234, max_depth = 15,
#                           n_estimators = 1000, max_features = 4)
# )
# rf_model_final.fit(train.loc[:, feature_names], train.loc[:, outcome_name])


# with open("/home/phil/Desktop/gradnet/trained/rf_model.pkl", "wb") as fh:
#     pickle.dump(rf_model_final, fh)

# GBR = GradientBoostingRegressor(n_estimators = 1000, max_depth = 2, random_state = 1234)
# GBR.fit(train.loc[:, feature_names], train.loc[:, outcome_name])

# with open("/home/phil/Desktop/gradnet/trained/gbr_model.pkl", "wb") as fh:
#     pickle.dump(GBR, fh)
        
# train.to_csv("processed/train_proportion.csv", header = True, index = False)
# test.to_csv("processed/test_proportion.csv", header = True, index = False)
